<a href="https://colab.research.google.com/github/malvikamurkumbi/QM2/blob/main/S2_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collecting data using Earth Engine API, importing required libraries

In [10]:
import ee
import pandas as pd
import numpy as np
from datetime import datetime

ee.Authenticate()
ee.Initialize(project='ee-malvikamurkumbi')

Setting a region of interest (10km around the centre of the dam)

In [2]:
roi = ee.Geometry.Point(60.79306,46.09667).buffer(10000)

Specifying the MODIS we want to extract images from

In [3]:
modis = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(roi)

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13Q1! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1

  warnings.warn(warning, category=DeprecationWarning)


Defining a function to extract pixel-level data, specifying the resolution (500m) and region of interest (previously defined)

In [4]:
def extract_pixel_ndvi(image):
    ndvi = image.select('NDVI')
    samples = ndvi.addBands(image.metadata('system:time_start','date')).sample(
        region=roi,
        scale=500,
        projection=ndvi.projection(),
        geometries=True
    )
    return samples

Extracting data year-by-year (since Google Earth Engine has a limit on the number of data points that can be extracted at once)

In [5]:
data = []

In [7]:
for year in range(2000, 2024):
    yearly_modis = modis.filterDate(str(year) + '-01-01', str(year) + '-12-31')

    ndvi_samples = yearly_modis.map(extract_pixel_ndvi).flatten()

    ndvi_data = ndvi_samples.toList(ndvi_samples.size()).getInfo()

    for sample in ndvi_data:
        ndvi_value = sample['properties']['NDVI']
        latitude = sample['geometry']['coordinates'][1]
        longitude = sample['geometry']['coordinates'][0]
        date = sample['properties']['date']
        data.append([date, ndvi_value, latitude, longitude])

Converting the extracted data to a pandas DataFrame

In [12]:
df = pd.DataFrame(data)
df

,0,1,2,3
0,950832000000,-134,46.182852,60.758907
1,950832000000,-145,46.182852,60.765402
2,950832000000,-139,46.182852,60.771897
3,950832000000,-141,46.182852,60.778391
4,950832000000,-117,46.182852,60.784886
...,...,...,...,...
1245207,1675296000000,138,46.011982,60.823428
1245208,1675296000000,203,46.011982,60.829903
1245209,1675296000000,182,46.007485,60.786114
1245210,1675296000000,165,46.007485,60.792588


Formatting the date correctly, rescaling the NDVI values, naming the columns, and dividing the values into a treatment vs control group

In [13]:
df['Date'] = pd.to_datetime(df[0], unit='ms')
df['NDVI'] = df[1]/10000
df['Latitude'] = df[2]
df['Longitude'] = df[3]
df = df.drop([0,1,2,3], axis=1)
df['Region'] = np.where(df['Latitude'] >= 46.09667, 'North', 'South')
df

,Date,NDVI,Latitude,Longitude,Region
0,2000-02-18,-0.0134,46.182852,60.758907,North
1,2000-02-18,-0.0145,46.182852,60.765402,North
2,2000-02-18,-0.0139,46.182852,60.771897,North
3,2000-02-18,-0.0141,46.182852,60.778391,North
4,2000-02-18,-0.0117,46.182852,60.784886,North
...,...,...,...,...,...
1245207,2023-02-02,0.0138,46.011982,60.823428,South
1245208,2023-02-02,0.0203,46.011982,60.829903,South
1245209,2023-02-02,0.0182,46.007485,60.786114,South
1245210,2023-02-02,0.0165,46.007485,60.792588,South


Converting the data to a csv file

In [14]:
df.to_csv('ndvi.csv')